In [ ]:
import matplotlib.pyplot as plt
import gym
import numpy as np
from ppo_torch_conv1d import Agent
from utils import plot_learning_curve
import uniform_gust_environment as u_gust_env

import time


if __name__ == '__main__':
    seed = 0 ##  (0-4 )
    speed = 15
    ep_steps = 200
    UvS = 'uniform'

    env = u_gust_env.mfc_gust_Env(steps_per_ep = ep_steps, seed=seed)
    uvs = 'uni'
    N = 20
    batch_size = 5
    n_epochs = 4
    alpha = 0.00003
    tap = '6tap'
    # File directory for saving training and controller data
    file = 'Data/speed'+str(speed)+'/uniform_comp/seed'+str(seed)+'/'+tap
    ppo_file = 'Apr_10_'+uvs+'_sp' +str(speed)+'_'+tap+'_seed'+str(seed)+'/'
    agent = Agent(n_actions=env.action_space.n, batch_size=batch_size, 
                    alpha=alpha, n_epochs=n_epochs, 
                    input_dims=env.observation_space.shape, 
                    fc1_dims=512, fc2_dims=512, chk_dir=file+'/ppo/'+ppo_file,
                    seed=seed)
    n_games = 1000

    figure_file = 'plots/mfc_gust.png'

    best_score = env.reward_range[0]
    score_history = []

    learn_iters = 0
    avg_score = 0
    n_steps = 0
    
    for i in range(n_games):
        ep_file = file+'/training/'+uvs+'_sp'+str(speed)+'_seed'+str(seed)+'_episode_'+str(i)
        observation = env.reset()
        obs_hist = np.zeros([len(observation)+2,ep_steps+1]) 
        obs_hist[:-2,0] = observation[:,-1] 
        done = False
        score = 0
        c_ep_steps = 1
        while not done:
            if not env.g_rotating:
                action, prob, val = agent.choose_action(observation)
                observation_, reward, done, lift, drag = env.step(action)
                score += reward

                agent.remember(observation, action, prob, val, reward, done)
                obs_hist[:-2,c_ep_steps] = observation_[:,-1] 
                obs_hist[-2:,c_ep_steps] = lift, drag
                observation = observation_
                c_ep_steps += 1
                n_steps+=1
            else:
                env.check_table()
                env.pause_for_timing()
            
            if n_steps % N == 0:
                agent.learn()
                learn_iters += 1
            
            
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        np.savetxt(ep_file+'_obs'+'.csv', obs_hist, delimiter =",")
        
        if i>100:
            if avg_score > best_score:
                best_score = avg_score
                agent.save_models()

        print('episode', i, 'score %.1f' % score, 'avg score %.1f' % avg_score,
                'time_steps', n_steps, 'learning_steps', learn_iters)
    x = [i+1 for i in range(len(score_history))]
    
    np.savetxt(ep_file+'_score'+'.csv', score_history, delimiter =",")
    agent.save_models(end=True)
    env.end()
    plot_learning_curve(x, score_history, figure_file)
  